# Using just TensorFlow Framework for IRIS dataset

### Step 1: Import the required libraries


In [614]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow

### Step 2: Load the `IRIS` dataset

In [615]:
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv' #import IRIS dataset
df = pd.read_csv(path, header=None)

In [616]:
df.head(2)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


In [617]:
headers = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"]
### Assign the headers to `df`
df.columns = headers

df.sample(2)

,sepal_length,sepal_width,petal_length,petal_width,species
82,5.8,2.7,3.9,1.2,Iris-versicolor
58,6.6,2.9,4.6,1.3,Iris-versicolor


In [618]:
### Import SKlearn libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [619]:
### Let us prepare the data - separating the features from the target
X, y = df.values[:,:-1], df.values[:,-1]

In [620]:
X[:2]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3.0, 1.4, 0.2]], dtype=object)

In [621]:
y[:2]

array(['Iris-setosa', 'Iris-setosa'], dtype=object)

In [622]:
X = X.astype('float32')

In [623]:
y = LabelEncoder().fit_transform(y)

In [624]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### Step 3: Split the `IRIS` dataset into `train` and `test`

In [625]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.30, random_state=42)

In [626]:
X_train.shape, X_test.shape

((105, 4), (45, 4))

### Step 4: Build the neural network model

In [627]:
### Dertermine the number of features
n_features = X_train.shape[1]
print('Number of features:',n_features)

### Determine the number of classes
n_classes = len(np.unique(y))
print('Number of classes:', n_classes)

Number of features: 4
Number of classes: 3


#### Step 4A: Define the model

**`CNN:`Convoluted Neural Network**

*   initialize the weights and biases for each layer
*   implement the forward pass


In [628]:
class CNNModel(tensorflow.Module):
  def __init__(self, num_h_l, num_f, num_o_c, num_n_list=[]):#initializes weights and biases for each layer

    ### initialize data for this object
    self.num_h_l = num_h_l # number of hidden layers
    self.num_f = num_f # number of features: input variables
    self.num_o_c = num_o_c # number of output classes: softmax output
    self.num_n_list = num_n_list # number of list of neurons at each hidden layers

    ### hidden layers should be in the range (1, 50) else default value is 1
    self.num_h_l = self.num_h_l if self.num_h_l > 0 and self.num_h_l < 51 else 1
    ### if the self.num_n_list is empty or having less values, populate it with a cap on num of neurons at (10, 20)
    self.num_n_list = self.num_n_list if len(self.num_n_list) == (self.num_h_l + 1) else list(np.random.randint(low = 10,high=20,size=self.num_h_l + 1))
    ### insert num of features at the beggining and num of output classes at the end of that list
    self.num_n_list.insert(0, self.num_f)
    self.num_n_list.append(self.num_o_c)
    print(self.num_n_list)

    for i in range(1, self.num_h_l + 3):
      w_var_name = 'self.w' + str(i)
      b_var_name = 'self.b' + str(i)
      print(w_var_name, b_var_name)
      w_var_value = "tensorflow.Variable(tensorflow.random.normal([{n1}, {n2}], stddev=0.1), name='{w_var_name}')".format(n1=self.num_n_list[i-1], n2=self.num_n_list[i], w_var_name=w_var_name[5:])
      b_var_value = "tensorflow.Variable(tensorflow.zeros([{n}]), name='{b_var_name}')".format(n=self.num_n_list[i], b_var_name=b_var_name[5:])
      exec(f"{w_var_name} = {w_var_value}")
      exec(f"{b_var_name} = {b_var_value}")
      print("executed these: ", w_var_name, w_var_value)
      print("executed these: ", b_var_name, b_var_value)

  def __call__(self, x):# to implement the forward pass

    import tensorflow
    ### represents weight and bias for the first layer
    ### ReLU (REctified Linear Unit) - an activation function
    # performing a matrix multiplication between input features and the weight matrix. Also adding the bias matrix after in the next using `tf.add`
    self.l_dict = {}
    self.l_dict["layer1"] = tensorflow.nn.relu(tensorflow.add(tensorflow.matmul(x,self.w1), self.b1))
    #layer1 = tensorflow.nn.relu(tensorflow.add(tensorflow.matmul(x,self.w1), self.b1))
    for i in range(2, self.num_h_l + 2):
      layer_name = 'layer' + str(i)
      prev_layer_name = 'layer' + str(i-1)
      curr_w_var_name = 'self.w' + str(i)
      curr_b_var_name = 'self.b' + str(i)
      self.l_dict[layer_name] = tensorflow.nn.relu(tensorflow.add(tensorflow.matmul(self.l_dict[prev_layer_name], eval(curr_w_var_name)), eval(curr_b_var_name)))

    last_w_var_name = 'self.w' + str(self.num_h_l + 2)
    last_b_var_name = 'self.b' + str(self.num_h_l + 2)
    last_layer_name = 'layer' + str(self.num_h_l + 1)
    output = tensorflow.add(tensorflow.matmul(self.l_dict[last_layer_name], eval(last_w_var_name)), eval(last_b_var_name))
    return output

#### Step 4B: Instantiate the model





In [629]:
model = CNNModel(11, n_features, n_classes)

ListWrapper([4, 19, 12, 19, 18, 19, 12, 19, 19, 13, 17, 13, 15, 3])
self.w1 self.b1
executed these:  self.w1 tensorflow.Variable(tensorflow.random.normal([4, 19], stddev=0.1), name='w1')
executed these:  self.b1 tensorflow.Variable(tensorflow.zeros([19]), name='b1')
self.w2 self.b2
executed these:  self.w2 tensorflow.Variable(tensorflow.random.normal([19, 12], stddev=0.1), name='w2')
executed these:  self.b2 tensorflow.Variable(tensorflow.zeros([12]), name='b2')
self.w3 self.b3
executed these:  self.w3 tensorflow.Variable(tensorflow.random.normal([12, 19], stddev=0.1), name='w3')
executed these:  self.b3 tensorflow.Variable(tensorflow.zeros([19]), name='b3')
self.w4 self.b4
executed these:  self.w4 tensorflow.Variable(tensorflow.random.normal([19, 18], stddev=0.1), name='w4')
executed these:  self.b4 tensorflow.Variable(tensorflow.zeros([18]), name='b4')
self.w5 self.b5
executed these:  self.w5 tensorflow.Variable(tensorflow.random.normal([18, 19], stddev=0.1), name='w5')
executed thes

In [630]:
loss_fn = tensorflow.losses.SparseCategoricalCrossentropy(from_logits=True) # Crossentropy loss for multi-class classification
optimizer = tensorflow.optimizers.Adam() #Adam optimizer for training

#### Step 4C: Training Steps

In [631]:
### Training Function
@tensorflow.function # compiles the function into a TensorFlow graph for efficiency
def train_step(x,y): #Performs one step of training
  with tensorflow.GradientTape() as tape: #records operations for automatic differentiation
    logits = model(x)   # raw output of the model
    loss = loss_fn(y, logits) #compute the loss
  gradients = tape.gradient(loss, model.trainable_variables) #gradients of the loss w.r.t model parameters --> weights, biases
  optimizer.apply_gradients(zip(gradients,model.trainable_variables)) #updates the model parameters --> weights, biases
  return loss

#### Step 4D: Run the model loop

In [632]:
X_train.shape

(105, 4)

In [633]:
n_epochs = 150 # no. of training epochs
batch_size = 32 # size of each training batch

for epoch in range(n_epochs): #looping over epochs
  # Creates a TF dataset from the training data,shuffles and bacthes it
  dataset = tensorflow.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1000).batch(batch_size)
  for batch_x, batch_y in dataset: #looping over batches
    loss = train_step(batch_x, batch_y) #refer the function train_step above
  if (epoch+1)%10 ==0:  #Print loss every 10 epochs
    print(f'Epoch {epoch + 1}, Loss: {loss.numpy()}')

Epoch 10, Loss: 1.0959783792495728
Epoch 20, Loss: 1.1034513711929321
Epoch 30, Loss: 1.0980970859527588
Epoch 40, Loss: 1.0804303884506226
Epoch 50, Loss: 1.1015653610229492
Epoch 60, Loss: 0.8544092178344727
Epoch 70, Loss: 0.8133599162101746
Epoch 80, Loss: 0.9818600416183472
Epoch 90, Loss: 0.4517137110233307
Epoch 100, Loss: 0.16860903799533844
Epoch 110, Loss: 0.35100454092025757
Epoch 120, Loss: 0.19981372356414795
Epoch 130, Loss: 0.1631355732679367
Epoch 140, Loss: 0.12904958426952362
Epoch 150, Loss: 0.10788355022668839


In [634]:
def evaluate(X,y): #function to compute accuracy on the test set
  logits = model(X) # raw outputs of the model
  predictions = tensorflow.argmax(logits, axis=1) #convers logits into predicted classes
  accuracy = np.mean(predictions.numpy()==y) #mean accuracy
  return accuracy

test_accuracy = evaluate(X_test, y_test)
print(f'Test Accuracy percentage %: {test_accuracy*100:.3f}')

Test Accuracy percentage %: 95.556
